<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Предварительные-операции" data-toc-modified-id="Предварительные-операции-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предварительные операции</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Создание-эмбеддингов-текстов" data-toc-modified-id="Создание-эмбеддингов-текстов-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Создание эмбеддингов текстов</a></span></li><li><span><a href="#Подготовка-выборок-с-признаками-и-целевым-признаком" data-toc-modified-id="Подготовка-выборок-с-признаками-и-целевым-признаком-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка выборок с признаками и целевым признаком</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Обучение-разных-моделей" data-toc-modified-id="Обучение-разных-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение разных моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Сравнение-моделей" data-toc-modified-id="Сравнение-моделей-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Сравнение моделей</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Определение токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель:** обучить модель классификации комментариев на позитивные и негативные со значением метрики качества *F1* не меньше 0,75.

Данные, на которых необходимо обучить модель: набор комментариев с разметкой токсичности правок (файл `toxic_comments.csv`: столбец *text* содержит текст комментария, а *toxic* — токсичность комментария, целевой признак).

## Подготовка

### Предварительные операции

Импортируем необходимые библиотеки.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tqdm import notebook
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
import torch

Прочитаем файл `toxic_comments.csv` и сохраним данные из файла в переменную `df`.

In [2]:
# сохранение данных в датафрейме
try:
    # Ссылка на датасет, сохраненный на домашнем ноутбуке
    df = pd.read_csv(
        'D:\\YandexDisk\\Data science\\Яндекс. Практикум\\Спринт 13. Машинное обучение для текстов\\Проект\\toxic_comments.csv',
        index_col=[0]
    )
except:
# Ссылка на датасет, сохраненный на сервере Яндекс
    df = pd.read_csv(
        '/datasets/toxic_comments.csv',
        index_col=[0]
    )

Рассмотрим первые строки датафрейма, чтобы получить первое впечатление о них.

In [3]:
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Рассмотрим общую информацию о датафрейме.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Рассчитаем долю пропусков в датафрейме.

In [5]:
df.isna().mean()

text     0.0
toxic    0.0
dtype: float64

Пропусков нет.

Рассмотрим статистические данные о датафрейме.

In [6]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
text,159292,159292,""", and welcome to Wikipedia! Thank you for you...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toxic,159292.0,NaN,NaN,NaN,0.101612,0.302139,0.0,0.0,0.0,0.0,1.0


В датафрейме содержатся данные типов `object` (столбец `text`) и `int64` (столбец `toxic`).

Столбец `text` содержит комментарии пользователей сервиса, а `toxic` - содержит данные о том, является ли комментарий допустимым или нет. Столбец `toxic` - целевой признак; значений класса `0` значительно больше, чем значений класса `1`.

#### Выводы

Предварительно можно сказать, что данных для выполнения проекта достаточно.

### Создание эмбеддингов текстов

Для уменьшения времени выполнения расчетов из исходного корпуса возьмем только часть текстов.

In [7]:
df = df.sample(n=3000, random_state=1).reset_index(drop=True)

Проверим достаточно ли в новой выборке объектов класса `1`.

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
toxic,3000.0,0.106333,0.308315,0.0,0.0,0.0,0.0,1.0


Для создания эмбеддингов используем предобученную модель RoBERTa (A Robustly Optimized BERT Pretraining Approach).

Инициализируем токенайзер.

In [9]:
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

Выполним токенизацию текста в столбце `text`.

In [10]:
%%time

# токенизация
df['encoded'] = df['text'].apply(
  lambda x: tokenizer.encode(x, add_special_tokens=True)).values

Token indices sequence length is longer than the specified maximum sequence length for this model (802 > 512). Running this sequence through the model will result in indexing errors


CPU times: total: 875 ms
Wall time: 2.91 s


Учитывая ограничение на длину текста модеди BERT, ограничим длину закодированного текста 512 токенами - оставим 256 токенов из начала и 256 символов из конца закодированного текста.

In [11]:
df['encoded_max_len_512'] = df['encoded'].apply(
  lambda x : x[:256]+x[len(x)-256:] if len(x) > 512 else x)

Определим наибольшую длину закодированного текста.

In [12]:
df['len'] = df['encoded_max_len_512'].apply(
  lambda x: len(x))

Для закодированного текста длиной меньше 512 токенов добавим нули для того, чтобы их длина равнялась 512 токенам.

In [13]:
n = df['len'].max()
df['padded'] = df['encoded_max_len_512'].apply(
  lambda x: x + [0]*(n - len(x)))

Создадим для закодированного текста маски внимания.

In [14]:
df['attention_mask'] = df['padded'].apply(
  lambda x: np.where(np.array(x) != 0, 1, 0))

Инициализируем модель RoBERTa.

In [15]:
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Выполним создание эмбеддингов

In [16]:
%time

batch_size = 100 # по умолчанию 100
embeddings = []

padded = np.vstack(df['padded'])
attention_mask = np.vstack(df['attention_mask'])

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(np.array(batch_embeddings[0]))

embeddings = np.concatenate(embeddings)

CPU times: total: 0 ns
Wall time: 0 ns


  0%|          | 0/30 [00:00<?, ?it/s]

### Подготовка выборок с признаками и целевым признаком

Разделим датафрейм на датафрейм, в котором содержатся признаки, и датафрейм, в котором содержатся целевые признаки.

In [17]:
features = embeddings
target = df['toxic']

Разделим датафрейм на обучающий, валидационный и тестовый.

In [18]:
features_train_valid, features_test, target_train_valid, target_test = (
    train_test_split(features, target, test_size=0.25, stratify=target)
)

### Выводы

Выполнена подготовка эмбеддингов для обучения моделей классификации и разделение выборки на обучающую, валидационную и тестовую.

## Обучение разных моделей

### Дерево решений

Обучим модель дерева решений и оценим значение *F1*.

In [25]:
%%time

def cross_val_f1(features, target, blocks, clf_model):
    
    f1_train_scores = []
    f1_valid_scores = []
    features = pd.DataFrame(features)
    target = target.reset_index(drop=True)
    
    sample_size = int(len(features)/blocks)
       
    for i in range(0, len(features), sample_size):
        
        train_indexes = pd.concat([features.iloc[:i], features.iloc[i + sample_size:]]).index
        valid_indexes = features.iloc[i: i + sample_size].index
        
        # разобьем переменные features и target на выборки features_train, target_train, features_valid, target_valid
        features_train = features.iloc[train_indexes].reset_index(drop=True)
        target_train = target.iloc[train_indexes].reset_index(drop=True)

        features_valid = features.iloc[valid_indexes].reset_index(drop=True)
        target_valid = target.iloc[valid_indexes].reset_index(drop=True)
        
        model = clf_model
        model = model.fit(features_train, target_train.values)
        
        f1_train_score = f1_score(target_train, model.predict(features_train))
        f1_valid_score = f1_score(target_valid, model.predict(features_valid))
        
        f1_train_scores.append(f1_train_score)
        f1_valid_scores.append(f1_valid_score)
    
    # рассчитаем среднее значение качества модели
    print('Среднее значение метрики f1 модели на обучающей выборке:', np.mean(f1_train_scores))
    print('Среднее значение метрики f1 модели на валидационной выборке:', np.mean(f1_valid_scores))
    
    return np.mean(f1_train_scores), np.mean(f1_valid_scores)
    
# выполнение функции
decision_tree_train_f1, decision_tree_valid_f1 = cross_val_f1(
    features_train_valid, target_train_valid, 3, DecisionTreeClassifier(class_weight='balanced', random_state=1))

Среднее значение метрики f1 модели на обучающей выборке: 1.0
Среднее значение метрики f1 модели на валидационной выборке: 0.753875901980163
CPU times: total: 15.6 ms
Wall time: 49 ms


### Случайный лес

Обучим модель случайного леса и оценим значение *f1*.

In [26]:
%%time

random_forest_train_f1, random_forest_valid_f1 = cross_val_f1(
    features_train_valid, target_train_valid, 3, RandomForestClassifier(random_state=1, class_weight='balanced', n_estimators=1000))

Среднее значение метрики f1 модели на обучающей выборке: 1.0
Среднее значение метрики f1 модели на валидационной выборке: 0.8049979211998167
CPU times: total: 3.66 s
Wall time: 7.27 s


### Логистическая регрессия

Обучим модель логистической регрессии и оценим значение *f1*.

In [27]:
%%time

logistic_regression_train_f1, logistic_regression_valid_f1 = cross_val_f1(
    features_train_valid, target_train_valid, 3, LogisticRegression(C=5, class_weight='balanced', solver='saga', max_iter=2000, penalty='l2', random_state=1))

Среднее значение метрики f1 модели на обучающей выборке: 0.7801064318523881
Среднее значение метрики f1 модели на валидационной выборке: 0.7865103441831284
CPU times: total: 93.8 ms
Wall time: 110 ms


### Сравнение моделей

Сведем собранные данные в таблицу.

In [28]:
pd.DataFrame({'ML model':['f1 (train) value', 'f1 (test) value'],
             'DecisionTreeClassifier':[decision_tree_train_f1, decision_tree_valid_f1],
             'RandomForestClassifier':[random_forest_train_f1, random_forest_valid_f1],
             'LogisticRegressionClassifier':[logistic_regression_train_f1, logistic_regression_valid_f1],
             }).set_index('ML model').T.sort_values(by = ['f1 (test) value'], ascending = False)

ML model,f1 (train) value,f1 (test) value
RandomForestClassifier,1.000000,0.804998
LogisticRegressionClassifier,0.780106,0.786510
DecisionTreeClassifier,1.000000,0.753876


Как мы видим, наилучшее значение *f1* получено для модели случайного леса.

### Выводы

Обучены 3 модели, наилучший результат получен для модели случайного леса

## Тестирование

Выполним определение метрики *f1* для модели случайного леса, обученной на обучающей и тестовой выборках.

In [29]:
best_model = RandomForestClassifier(random_state=1, class_weight='balanced', n_estimators=1000)
best_model.fit(features_train_valid, target_train_valid)

f1_train_score = f1_score(target_train_valid, best_model.predict(features_train_valid))
f1_valid_score = f1_score(target_test, best_model.predict(features_test))

print(f'Значение f1 на обучающей выборке : {f1_train_score}')
print(f'Значение f1 на тестовой выборке: {f1_valid_score}')

Значение f1 на обучающей выборке : 1.0
Значение f1 на тестовой выборке: 0.7651006711409397


## Выводы

В предварительной части были подготовлены эмбеддинги из части текстов исходного корпуса.

Обучены 3 модели классификации:
* дерево решений;
* случайный лес;
* логистическая регрессия.

Исходя из критерия задания (наименьшее значение *f1*, не больше 0,75), выбрана наилучшая модель - модель случайного леса.

Можно рекомендовать данную модель для выявления токсичных комментариев.